In [1]:
# !pip install numpy
# !pip install pandas
# !pip install tqdm
# !pip install sklearn
# !pip install matplotlib

In [10]:
#This is dev branch
import datetime
import math
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import multiprocessing as mp
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor

NUM_TIME_SLOTS = 144
NUM_DAYS_IN_DATA = 0

In [11]:
# 24 hours is divided into 144 slots where each slot is 10 mins long
def calculateTimeSlot(time,printValue=True):
    global NUM_TIME_SLOTS
    dateTime = datetime.datetime.strptime(time, '%Y-%m-%d %H:%M:%S')
    timePart = dateTime.time()
    timeInMinutes = (timePart.hour * 60) + timePart.minute + (timePart.second/60) + 1
    timeSlot = timeInMinutes/10
    roundedTimeSlot = math.ceil(timeSlot)
    if roundedTimeSlot > NUM_TIME_SLOTS:
        roundedTimeSlot -= 1
    if printValue==True:
        print(f"time: {time} timeInMinutes: {timeInMinutes} timeSlot: {roundedTimeSlot}")
    return int(roundedTimeSlot)

def extractDayOfWeek(time):
    d = datetime.datetime.strptime(time, '%Y-%m-%d %H:%M:%S')
    return d.weekday()

def extractDate(time):
    d = datetime.datetime.strptime(time, '%Y-%m-%d %H:%M:%S')
    return str(d.date())

print(type(extractDate('2019-01-01 00:00:00')))

<class 'str'>


In [12]:
def extractNumberOfPOI(poiArray):
    poiCount = 0
    for poiEntry in poiArray:
        poiEntry = poiEntry + ':'
        divisions = poiEntry.split(":")
        poiCount += int(divisions[1])
    return poiCount
        

In [13]:
def readMultipleData(path,fileNamePrefix,headerNames,dataTypes,delimiter2="\t"):
    global NUM_DAYS_IN_DATA
    filesToExplore = []
    for file in os.listdir(path):
        if file.startswith(fileNamePrefix):
            filesToExplore.append(file)
            # print(f"{file} read")
        else:
            continue

    print(f"{len(filesToExplore)} files read")
    if fileNamePrefix == 'order':
        NUM_DAYS_IN_DATA = len(filesToExplore)
    
    readData = []
    for files in filesToExplore:
        fileRead = pd.read_csv(path + files, sep=delimiter2, names=headerNames,dtype=dataTypes)
        readData.append(fileRead)

    readData = pd.concat(readData, ignore_index=True)
    return readData

In [14]:
# now region Data
regionData = pd.read_csv('./training_data/cluster_map/cluster_map', sep='\t', names=['region_hash', 'region_id'],dtype={'region_hash': 'str', 'region_id': 'int'})
# print(regionData.head())
# regionData.to_csv('regionData.csv',index=False)

In [ ]:
def getRegionId(regionHash):
    for _, row in regionData.iterrows():
        if row['region_hash'] == regionHash:
            return row['region_id']

In [15]:
# read order data
dataTypes = {'order_id':'str', 'driver_id':'str', 'passenger_id':'str', 'start_region_hash':'str', 'dest_region_hash':'str', 'price':'double', 'time':'str'}
orderDataPath = './training_data/order_data/'
orderData = readMultipleData(orderDataPath,'order', ['order_id', 'driver_id', 'passenger_id', 'start_region_hash', 'dest_region_hash', 'price', 'time'], dataTypes)
print("printing order data")
# print(orderData.head())


21 files read
printing order data


In [16]:
# read weather data
dataTypes={'time':'str', 'weather':'int', 'temperature':'double', 'PM2.5':'double'}
weatherDataPath = './training_data/weather_data/'
weatherData = readMultipleData(weatherDataPath,'weather', ['time', 'weather', 'temperature', 'PM2.5'], dataTypes)
print("printing weather data")
# print(weatherData.head())

21 files read
printing weather data


In [17]:
weatherData['time_slot'] = weatherData['time'].apply(calculateTimeSlot,printValue=False)
weatherData['date'] = weatherData['time'].apply(extractDate)
# weatherData['day_of_week'] = weatherData['time'].apply(extractDayOfWeek)
weatherData = weatherData.drop(['temperature','PM2.5','time'], axis=1)
print(weatherData.head())

   weather  time_slot        date
0        1          1  2016-01-01
1        1          1  2016-01-01
2        1          2  2016-01-01
3        1          2  2016-01-01
4        1          3  2016-01-01


In [18]:
orderData = pd.merge(regionData,orderData, how='left', right_on='start_region_hash', left_on='region_hash')
orderData['time_slot'] = orderData['time'].apply(calculateTimeSlot,printValue=False)
orderData['day_of_week'] = orderData['time'].apply(extractDayOfWeek)
orderData['date'] = orderData['time'].apply(extractDate)
orderData = orderData.drop(['passenger_id', 'dest_region_hash','start_region_hash','price'], axis=1)
print(orderData.head())
# regionData=None

                        region_hash  region_id   
0  90c5a34f06ac86aee0fd70e2adce7d8a          1  \
1  90c5a34f06ac86aee0fd70e2adce7d8a          1   
2  90c5a34f06ac86aee0fd70e2adce7d8a          1   
3  90c5a34f06ac86aee0fd70e2adce7d8a          1   
4  90c5a34f06ac86aee0fd70e2adce7d8a          1   

                           order_id                         driver_id   
0  1654babc363bc6d0f5d01fc0bafedc1a  cc26812d679c9e55a6bf63eed315e989  \
1  5ac4ac8d0e6092ea1dc323d367613ffd  f6c760be3cd8521c612657da7788f9dc   
2  ce86a6ae5eee7a2ea954323b6c01510b  360478560b1fd4b3eb757074c91ee709   
3  b5e816c08e44565c7ed67a6f6e366708  0359fc335d238c6206703d1d7e3620c8   
4  25ed10b13aaa36071deecab3aa374be3  c03944aff7444c27fd7b04cdd3e80af5   

                  time  time_slot  day_of_week        date  
0  2016-01-01 20:49:15        126            4  2016-01-01  
1  2016-01-01 13:04:32         79            4  2016-01-01  
2  2016-01-01 21:00:21        127            4  2016-01-01  
3  2016-01-01 19

In [19]:
# orderData.to_csv('orderData.csv',index=False)

In [20]:
# orderData = pd.read_csv('orderData.csv')
# print(orderData.head())
# print(orderData.shape)

In [21]:
# read POI Data
poiDataStr = {
    'region_hash':[],
    'poi_class':[]
}
with open('./training_data/poi_data/poi_data','r') as fileToRead:
    for line in fileToRead:
        line = line.strip()
        columns = line.split('\t')
        poiDataStr['region_hash'].append(columns[0])
        remData = columns[1:]
        poiDataStr['poi_class'].append(remData)
        
poiData = pd.DataFrame(poiDataStr,columns=['region_hash','poi_class'])
poiData['poi_count'] = poiData['poi_class'].apply(extractNumberOfPOI)
poiData = pd.merge(regionData,poiData, how='inner', on='region_hash')
poiData = poiData.drop(['region_hash'], axis=1)
poiData = poiData.drop(['poi_class'], axis=1)
print(poiData.head())

   region_id  poi_count
0          1     653376
1          2     343537
2          3      31125
3          4     187829
4          5      27888


In [22]:
# print(mergedDataCSV)
orderData['requests'] = 1
# print(orderData.head())
groupedMergedData = orderData.groupby(['region_id','time_slot','day_of_week'])['requests'].agg('sum').reset_index()
# groupedMergedData = groupedMergedData.drop(['date','region_hash','order_id','driver_id','time'], axis=1)
orderData = pd.merge(orderData,groupedMergedData, how='left' , on=['region_id','time_slot','day_of_week'])
print(orderData.head())

                        region_hash  region_id   
0  90c5a34f06ac86aee0fd70e2adce7d8a          1  \
1  90c5a34f06ac86aee0fd70e2adce7d8a          1   
2  90c5a34f06ac86aee0fd70e2adce7d8a          1   
3  90c5a34f06ac86aee0fd70e2adce7d8a          1   
4  90c5a34f06ac86aee0fd70e2adce7d8a          1   

                           order_id                         driver_id   
0  1654babc363bc6d0f5d01fc0bafedc1a  cc26812d679c9e55a6bf63eed315e989  \
1  5ac4ac8d0e6092ea1dc323d367613ffd  f6c760be3cd8521c612657da7788f9dc   
2  ce86a6ae5eee7a2ea954323b6c01510b  360478560b1fd4b3eb757074c91ee709   
3  b5e816c08e44565c7ed67a6f6e366708  0359fc335d238c6206703d1d7e3620c8   
4  25ed10b13aaa36071deecab3aa374be3  c03944aff7444c27fd7b04cdd3e80af5   

                  time  time_slot  day_of_week        date  requests_x   
0  2016-01-01 20:49:15        126            4  2016-01-01           1  \
1  2016-01-01 13:04:32         79            4  2016-01-01           1   
2  2016-01-01 21:00:21        127     

In [23]:
orderData = orderData.drop(["requests_x"], axis=1)
orderData = orderData.rename(columns={"requests_y": "requests"})
print(orderData)

                              region_hash  region_id   
0        90c5a34f06ac86aee0fd70e2adce7d8a          1  \
1        90c5a34f06ac86aee0fd70e2adce7d8a          1   
2        90c5a34f06ac86aee0fd70e2adce7d8a          1   
3        90c5a34f06ac86aee0fd70e2adce7d8a          1   
4        90c5a34f06ac86aee0fd70e2adce7d8a          1   
...                                   ...        ...   
8540609  1ecbb52d73c522f184a6fc53128b1ea1         66   
8540610  1ecbb52d73c522f184a6fc53128b1ea1         66   
8540611  1ecbb52d73c522f184a6fc53128b1ea1         66   
8540612  1ecbb52d73c522f184a6fc53128b1ea1         66   
8540613  1ecbb52d73c522f184a6fc53128b1ea1         66   

                                 order_id                         driver_id   
0        1654babc363bc6d0f5d01fc0bafedc1a  cc26812d679c9e55a6bf63eed315e989  \
1        5ac4ac8d0e6092ea1dc323d367613ffd  f6c760be3cd8521c612657da7788f9dc   
2        ce86a6ae5eee7a2ea954323b6c01510b  360478560b1fd4b3eb757074c91ee709   
3        b5

In [24]:
orderData['temp'] = 1
orderData['answers'] = orderData['temp'].where(orderData['driver_id'].notnull(), 0)
orderData = orderData.drop(['temp'], axis=1)
groupedMergedData = orderData.groupby(['region_id','time_slot','day_of_week'])['answers'].agg('sum').reset_index()
orderData = pd.merge(orderData,groupedMergedData, how='left' , on=['region_id','time_slot','day_of_week'])
print(orderData)

                              region_hash  region_id   
0        90c5a34f06ac86aee0fd70e2adce7d8a          1  \
1        90c5a34f06ac86aee0fd70e2adce7d8a          1   
2        90c5a34f06ac86aee0fd70e2adce7d8a          1   
3        90c5a34f06ac86aee0fd70e2adce7d8a          1   
4        90c5a34f06ac86aee0fd70e2adce7d8a          1   
...                                   ...        ...   
8540609  1ecbb52d73c522f184a6fc53128b1ea1         66   
8540610  1ecbb52d73c522f184a6fc53128b1ea1         66   
8540611  1ecbb52d73c522f184a6fc53128b1ea1         66   
8540612  1ecbb52d73c522f184a6fc53128b1ea1         66   
8540613  1ecbb52d73c522f184a6fc53128b1ea1         66   

                                 order_id                         driver_id   
0        1654babc363bc6d0f5d01fc0bafedc1a  cc26812d679c9e55a6bf63eed315e989  \
1        5ac4ac8d0e6092ea1dc323d367613ffd  f6c760be3cd8521c612657da7788f9dc   
2        ce86a6ae5eee7a2ea954323b6c01510b  360478560b1fd4b3eb757074c91ee709   
3        b5

In [25]:
orderData = orderData.drop(["answers_x"], axis=1)
orderData = orderData.rename(columns={"answers_y": "answers"})

In [26]:
orderData['supply_demand'] = orderData['requests'] - orderData['answers']
print(orderData)

                              region_hash  region_id   
0        90c5a34f06ac86aee0fd70e2adce7d8a          1  \
1        90c5a34f06ac86aee0fd70e2adce7d8a          1   
2        90c5a34f06ac86aee0fd70e2adce7d8a          1   
3        90c5a34f06ac86aee0fd70e2adce7d8a          1   
4        90c5a34f06ac86aee0fd70e2adce7d8a          1   
...                                   ...        ...   
8540609  1ecbb52d73c522f184a6fc53128b1ea1         66   
8540610  1ecbb52d73c522f184a6fc53128b1ea1         66   
8540611  1ecbb52d73c522f184a6fc53128b1ea1         66   
8540612  1ecbb52d73c522f184a6fc53128b1ea1         66   
8540613  1ecbb52d73c522f184a6fc53128b1ea1         66   

                                 order_id                         driver_id   
0        1654babc363bc6d0f5d01fc0bafedc1a  cc26812d679c9e55a6bf63eed315e989  \
1        5ac4ac8d0e6092ea1dc323d367613ffd  f6c760be3cd8521c612657da7788f9dc   
2        ce86a6ae5eee7a2ea954323b6c01510b  360478560b1fd4b3eb757074c91ee709   
3        b5

In [27]:
orderData = orderData.drop(['order_id','driver_id','requests','answers'], axis=1)
# orderData = m.drop_duplicates(subset=['region_id','time_slot','day_of_week'], keep='first')
print(orderData) # here

                              region_hash  region_id                 time   
0        90c5a34f06ac86aee0fd70e2adce7d8a          1  2016-01-01 20:49:15  \
1        90c5a34f06ac86aee0fd70e2adce7d8a          1  2016-01-01 13:04:32   
2        90c5a34f06ac86aee0fd70e2adce7d8a          1  2016-01-01 21:00:21   
3        90c5a34f06ac86aee0fd70e2adce7d8a          1  2016-01-01 19:25:32   
4        90c5a34f06ac86aee0fd70e2adce7d8a          1  2016-01-01 20:35:36   
...                                   ...        ...                  ...   
8540609  1ecbb52d73c522f184a6fc53128b1ea1         66  2016-01-21 11:41:43   
8540610  1ecbb52d73c522f184a6fc53128b1ea1         66  2016-01-21 21:41:23   
8540611  1ecbb52d73c522f184a6fc53128b1ea1         66  2016-01-21 15:14:10   
8540612  1ecbb52d73c522f184a6fc53128b1ea1         66  2016-01-21 09:29:42   
8540613  1ecbb52d73c522f184a6fc53128b1ea1         66  2016-01-21 18:07:27   

         time_slot  day_of_week        date  supply_demand  
0             

In [28]:
# merge poi data here
orderData = pd.merge(orderData,poiData, how="inner", on='region_id')
print(orderData)

                              region_hash  region_id                 time   
0        90c5a34f06ac86aee0fd70e2adce7d8a          1  2016-01-01 20:49:15  \
1        90c5a34f06ac86aee0fd70e2adce7d8a          1  2016-01-01 13:04:32   
2        90c5a34f06ac86aee0fd70e2adce7d8a          1  2016-01-01 21:00:21   
3        90c5a34f06ac86aee0fd70e2adce7d8a          1  2016-01-01 19:25:32   
4        90c5a34f06ac86aee0fd70e2adce7d8a          1  2016-01-01 20:35:36   
...                                   ...        ...                  ...   
8540609  1ecbb52d73c522f184a6fc53128b1ea1         66  2016-01-21 11:41:43   
8540610  1ecbb52d73c522f184a6fc53128b1ea1         66  2016-01-21 21:41:23   
8540611  1ecbb52d73c522f184a6fc53128b1ea1         66  2016-01-21 15:14:10   
8540612  1ecbb52d73c522f184a6fc53128b1ea1         66  2016-01-21 09:29:42   
8540613  1ecbb52d73c522f184a6fc53128b1ea1         66  2016-01-21 18:07:27   

         time_slot  day_of_week        date  supply_demand  poi_count  
0  

In [29]:
# orderData['date'] = orderData['date'].apply(lambda x: str(x))
# weatherData['date'] = weatherData['date'].apply(lambda x: str(x))
# print(weatherData['date'][0]==orderData['date'][0])
# print(orderData['date'][0])

In [30]:
# merge weather data here
orderData = pd.merge(weatherData,orderData, how="inner", on=['date','time_slot'])
print(orderData)

          weather  time_slot        date                       region_hash   
0               1          1  2016-01-01  90c5a34f06ac86aee0fd70e2adce7d8a  \
1               1          1  2016-01-01  90c5a34f06ac86aee0fd70e2adce7d8a   
2               1          1  2016-01-01  90c5a34f06ac86aee0fd70e2adce7d8a   
3               1          1  2016-01-01  90c5a34f06ac86aee0fd70e2adce7d8a   
4               1          1  2016-01-01  90c5a34f06ac86aee0fd70e2adce7d8a   
...           ...        ...         ...                               ...   
12222999        2        144  2016-01-21  1ecbb52d73c522f184a6fc53128b1ea1   
12223000        2        144  2016-01-21  1ecbb52d73c522f184a6fc53128b1ea1   
12223001        2        144  2016-01-21  1ecbb52d73c522f184a6fc53128b1ea1   
12223002        2        144  2016-01-21  1ecbb52d73c522f184a6fc53128b1ea1   
12223003        2        144  2016-01-21  1ecbb52d73c522f184a6fc53128b1ea1   

          region_id                 time  day_of_week  supply_d

In [31]:
m = orderData.drop_duplicates(subset=['region_id','time_slot','day_of_week','weather','poi_count'], keep='first')
print(m)

          weather  time_slot        date                       region_hash   
0               1          1  2016-01-01  90c5a34f06ac86aee0fd70e2adce7d8a  \
165             1          1  2016-01-01  f2c8c4bb99e6377d21de71275afd6cd2   
174             1          1  2016-01-01  58c7a4888306d8ff3a641d1c0feccbe3   
177             1          1  2016-01-01  b26a240205c852804ff8758628c0a86a   
200             1          1  2016-01-01  1cbfbdd079ef93e74405c53fcfff8567   
...           ...        ...         ...                               ...   
12221491        2        144  2016-01-21  d524868ce69cb9db10fc5af177fb9423   
12221492        2        144  2016-01-21  de092beab9305613aca8f79d7d7224e7   
12221495        2        144  2016-01-21  bf44d327f0232325c6d5280926d7b37d   
12221508        2        144  2016-01-21  825a21aa308dea206adb49c4b77c7805   
12221510        2        144  2016-01-21  1ecbb52d73c522f184a6fc53128b1ea1   

          region_id                 time  day_of_week  supply_d

<h1 style="background-color:green">Train Model</h1>

In [32]:
# Apply model here
X = m[['region_id','time_slot','day_of_week','weather','poi_count']]
Y = m['supply_demand']

# RandomForestRegressor
# Xtrain, Xtest, Ytrain, Ytest = train_test_split(X,Y, test_size=0.1, random_state=42)
# AImodel = RandomForestRegressor(n_estimators=100, max_depth=8, random_state=42)
# AImodel.fit(Xtrain,Ytrain)

# DecisionTreeRegressor
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X,Y, test_size=0.3, random_state=42)
AImodel = DecisionTreeRegressor(max_depth=40)
AImodel.fit(Xtrain,Ytrain)

# gradientBoostingRegressor
# Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2, random_state=42)
# AImodel = GradientBoostingRegressor(n_estimators=100, max_depth=8, learning_rate=0.1)
# AImodel.fit(X, Y)



DecisionTreeRegressor(max_depth=40)

In [33]:
print("Accuracy",AImodel.score(Xtest,Ytest)) # score on test split

Accuracy 0.9563922778787027


In [34]:
# make rolling window function
def rollTheWindow(data,windowSize):
    datasetLength = len(data)
    listOfPredictions = []
    numberOfWindows = datasetLength//windowSize
    index = 0
    for i in range(numberOfWindows):
        rollingWindow = []
        lowerLimit = index
        upperLimit = index+windowSize
        rollingWindow.append(data[lowerLimit:upperLimit])
        rollingWindow = pd.concat(rollingWindow)
        X_WINDOW_INPUT = rollingWindow[['region_id','time_slot','day_of_week','weather','poi_count']]
        Y_WINDOW_OUTPUT = rollingWindow['supply_demand']
        currentPrediction = AImodel.predict(X_WINDOW_INPUT)
        meanSqError = mean_squared_error(Y_WINDOW_OUTPUT, currentPrediction)
        print(f"Prediction for window [{lowerLimit},{upperLimit}] MSE={meanSqError}")
        predictionTuple = (lowerLimit,upperLimit,currentPrediction,meanSqError)
        listOfPredictions.append(predictionTuple)
        index = upperLimit + i
        if index >= datasetLength:
            break
    return listOfPredictions

In [35]:
listOfPredictions =  rollTheWindow(orderData,1000)
maxError= max(listOfPredictions, key=lambda x: x[3])
minEror = min(listOfPredictions, key=lambda x: x[3])
print(f"Max error is {maxError[3]} for window {maxError[0]} to {maxError[1]}")
print(f"Min error is {minEror[3]} for window {minEror[0]} to {minEror[1]}")

Prediction for window [0,1000] MSE=6.032
Prediction for window [1000,2000] MSE=24.312
Prediction for window [2001,3001] MSE=6.032
Prediction for window [3003,4003] MSE=21.756
Prediction for window [4006,5006] MSE=2.556
Prediction for window [5010,6010] MSE=7.347
Prediction for window [6015,7015] MSE=137.749
Prediction for window [7021,8021] MSE=440.817
Prediction for window [8028,9028] MSE=2.141
Prediction for window [9036,10036] MSE=5.439
Prediction for window [10045,11045] MSE=365.249
Prediction for window [11055,12055] MSE=203.317
Prediction for window [12066,13066] MSE=0.521
Prediction for window [13078,14078] MSE=2.902
Prediction for window [14091,15091] MSE=2.073
Prediction for window [15105,16105] MSE=0.116
Prediction for window [16120,17120] MSE=0.0
Prediction for window [17136,18136] MSE=0.037
Prediction for window [18153,19153] MSE=2.902
Prediction for window [19171,20171] MSE=2.073
Prediction for window [20190,21190] MSE=0.116
Prediction for window [21210,22210] MSE=0.0
Pred

In [36]:
print("Accuracy",AImodel.score(X,Y)) # score on whole data

Accuracy 0.9858548362947256


<h1 style="background-color:blue">Read Test Data</h1>

In [37]:
# read test data
# read region Data
regionTestData = pd.read_csv('./test_set/cluster_map/cluster_map', sep='\t', names=['region_hash', 'region_id'],dtype={'region_hash': 'str', 'region_id': 'int'})
print(regionTestData.head())
#regionData.to_csv('regionData.csv',index=False)

                        region_hash  region_id
0  90c5a34f06ac86aee0fd70e2adce7d8a          1
1  f2c8c4bb99e6377d21de71275afd6cd2          2
2  58c7a4888306d8ff3a641d1c0feccbe3          3
3  b26a240205c852804ff8758628c0a86a          4
4  4b9e4cf2fbdc8281b8a1f9f12b80ce4d          5


In [38]:
# read order data
dataTypes = {'order_id':'str', 'driver_id':'str','start_region_hash':'str', 'dest_region_hash':'str' ,'time':'str'}
orderDataPath = './test_set/order_data/'
orderTestData = readMultipleData(orderDataPath,'test_order', ['order_id', 'driver_id', 'start_region_hash','dest_region_hash','time'], dataTypes,delimiter2=",")
print("printing order data")
print(orderTestData.head())

5 files read
printing order data
                           order_id                         driver_id   
0  a903b5f7f65f1dc7f4ee94fec74673be  dce1e90fc91ed39a7b04a22d02910a7d  \
1  b7c838beaf12a2132776a1f00e016038  1f39eba5ce330f20c95177826a122e12   
2  0e0d3c93298ed59281352e34c6f1ec5a  8e4e2bc0342b3c55edea2723f6613e36   
3  1f6d0d7f68f216c4333969d6152a0a8b  5a33393e516673c8e9a065915667a30f   
4  bcf6050d9f5b270f8beb3ff80b01b435  f0232b8e45abc5ca92ca0f90fa811e7c   

                  start_region_hash                  dest_region_hash   
0  d4ec2125aff74eded207d2d915ef682f  d4ec2125aff74eded207d2d915ef682f  \
1  2407d482f0ffa22a947068f2551fe62c  2407d482f0ffa22a947068f2551fe62c   
2  b26a240205c852804ff8758628c0a86a  3a43dcdff3c0b66b1acb1644ff055f9d   
3  4725c39a5e5f4c188d382da3910b3f3f  4725c39a5e5f4c188d382da3910b3f3f   
4  dd8d3b9665536d6e05b29c2648c0e69a  a5609739c6b5c2719a3752327c5e33a7   

                  time  
0  2016-01-23 07:20:52  
1  2016-01-23 13:08:17  
2  2016-01-23 

In [39]:
# read weather data
dataTypes={'time':'str', 'weather':'int', 'temperature':'double', 'PM2.5':'double'}
weatherTestDataPath = './test_set/weather_data/'
weatherTestData = readMultipleData(weatherTestDataPath,'weather_data', ['time', 'weather', 'temperature', 'PM2.5'], dataTypes)
print("printing weather data")
print(weatherTestData.head())

5 files read
printing weather data
                  time  weather  temperature  PM2.5
0  2016-01-23 07:01:12        4          1.0   94.0
1  2016-01-23 11:14:04        3         -1.0  107.0
2  2016-01-23 13:16:44        3         -1.0   56.0
3  2016-01-23 15:02:02        3          0.0   48.0
4  2016-01-23 15:16:39        3          0.0   48.0


In [40]:
#pre processing weather Test Data based on weather timeSlot 
weatherTestData['time_slot'] = weatherTestData['time'].apply(calculateTimeSlot,printValue=False)
weatherTestData['date'] = weatherTestData['time'].apply(extractDate)
# weatherData['day_of_week'] = weatherData['time'].apply(extractDayOfWeek)
weatherTestData = weatherTestData.drop(['temperature','PM2.5','time'], axis=1)
print(weatherTestData.head())

   weather  time_slot        date
0        4         43  2016-01-23
1        3         68  2016-01-23
2        3         80  2016-01-23
3        3         91  2016-01-23
4        3         92  2016-01-23


In [41]:
orderTestData = pd.merge(regionTestData,orderTestData, how='inner', right_on='start_region_hash', left_on='region_hash')
orderTestData['time_slot'] = orderTestData['time'].apply(calculateTimeSlot,printValue=False)
orderTestData['day_of_week'] = orderTestData['time'].apply(extractDayOfWeek)
orderTestData['date'] = orderTestData['time'].apply(extractDate)
orderTestData = orderTestData.drop(['dest_region_hash','start_region_hash'], axis=1)
print(orderTestData.head())
# regionData=None

                        region_hash  region_id   
0  90c5a34f06ac86aee0fd70e2adce7d8a          1  \
1  90c5a34f06ac86aee0fd70e2adce7d8a          1   
2  90c5a34f06ac86aee0fd70e2adce7d8a          1   
3  90c5a34f06ac86aee0fd70e2adce7d8a          1   
4  90c5a34f06ac86aee0fd70e2adce7d8a          1   

                           order_id                         driver_id   
0  31bcca3b90b4d971697156b63900d514  172b54135139345fb9d24d0b23635387  \
1  60d38d302c9689bcd2a31cf236b36c64  9db845353e2256db7ef3d54872d2288a   
2  cca7aca7134cffb46957d59897dcdbbc  773dd49e4148d807b099b42372fd5ac7   
3  85cbab6eaea4b6bf1bc2cd7ce6701a93  75eb96caaa6bacb30b810996d655dd54   
4  d1ee089ee022df88bb35e9224cb2ef0a  cf216f7293f44f4380301036b69acfa3   

                  time  time_slot  day_of_week        date  
0  2016-01-23 19:09:39        116            5  2016-01-23  
1  2016-01-23 19:29:47        118            5  2016-01-23  
2  2016-01-23 21:03:43        127            5  2016-01-23  
3  2016-01-23 07

In [42]:
# read POI Data
poiDataStr = {
    'region_hash':[],
    'poi_class':[]
}
with open('./test_set/poi_data/poi_data','r') as fileToRead:
    for line in fileToRead:
        line = line.strip()
        columns = line.split('\t')
        poiDataStr['region_hash'].append(columns[0])
        remData = columns[1:]
        poiDataStr['poi_class'].append(remData)
        
poiTestData = pd.DataFrame(poiDataStr,columns=['region_hash','poi_class'])
poiTestData['poi_count'] = poiTestData['poi_class'].apply(extractNumberOfPOI)
poiTestData = pd.merge(regionTestData,poiTestData, how='inner', on='region_hash')
poiTestData = poiTestData.drop(['region_hash'], axis=1)
poiTestData = poiTestData.drop(['poi_class'], axis=1)
print(poiTestData.head())

   region_id  poi_count
0          1     653376
1          2     343537
2          3      31125
3          4     187829
4          5      27888


In [43]:
# print(mergedDataCSV)
orderTestData['requests'] = 1
# print(orderData.head())
groupedTestMergedData = orderTestData.groupby(['region_id','time_slot','day_of_week'])['requests'].agg('sum').reset_index()
# groupedMergedData = groupedMergedData.drop(['date','region_hash','order_id','driver_id','time'], axis=1)
orderTestData = pd.merge(orderTestData,groupedTestMergedData, how='left' , on=['region_id','time_slot','day_of_week'])
print(orderTestData.head())

                        region_hash  region_id   
0  90c5a34f06ac86aee0fd70e2adce7d8a          1  \
1  90c5a34f06ac86aee0fd70e2adce7d8a          1   
2  90c5a34f06ac86aee0fd70e2adce7d8a          1   
3  90c5a34f06ac86aee0fd70e2adce7d8a          1   
4  90c5a34f06ac86aee0fd70e2adce7d8a          1   

                           order_id                         driver_id   
0  31bcca3b90b4d971697156b63900d514  172b54135139345fb9d24d0b23635387  \
1  60d38d302c9689bcd2a31cf236b36c64  9db845353e2256db7ef3d54872d2288a   
2  cca7aca7134cffb46957d59897dcdbbc  773dd49e4148d807b099b42372fd5ac7   
3  85cbab6eaea4b6bf1bc2cd7ce6701a93  75eb96caaa6bacb30b810996d655dd54   
4  d1ee089ee022df88bb35e9224cb2ef0a  cf216f7293f44f4380301036b69acfa3   

                  time  time_slot  day_of_week        date  requests_x   
0  2016-01-23 19:09:39        116            5  2016-01-23           1  \
1  2016-01-23 19:29:47        118            5  2016-01-23           1   
2  2016-01-23 21:03:43        127     

In [44]:
orderTestData = orderTestData.drop(["requests_x"], axis=1)
orderTestData = orderTestData.rename(columns={"requests_y": "requests"})
print(orderTestData.head())

                        region_hash  region_id   
0  90c5a34f06ac86aee0fd70e2adce7d8a          1  \
1  90c5a34f06ac86aee0fd70e2adce7d8a          1   
2  90c5a34f06ac86aee0fd70e2adce7d8a          1   
3  90c5a34f06ac86aee0fd70e2adce7d8a          1   
4  90c5a34f06ac86aee0fd70e2adce7d8a          1   

                           order_id                         driver_id   
0  31bcca3b90b4d971697156b63900d514  172b54135139345fb9d24d0b23635387  \
1  60d38d302c9689bcd2a31cf236b36c64  9db845353e2256db7ef3d54872d2288a   
2  cca7aca7134cffb46957d59897dcdbbc  773dd49e4148d807b099b42372fd5ac7   
3  85cbab6eaea4b6bf1bc2cd7ce6701a93  75eb96caaa6bacb30b810996d655dd54   
4  d1ee089ee022df88bb35e9224cb2ef0a  cf216f7293f44f4380301036b69acfa3   

                  time  time_slot  day_of_week        date  requests  
0  2016-01-23 19:09:39        116            5  2016-01-23       220  
1  2016-01-23 19:29:47        118            5  2016-01-23        24  
2  2016-01-23 21:03:43        127            5 

In [45]:
orderTestData['temp'] = 1
orderTestData['answers'] = orderTestData['temp'].where(orderTestData['driver_id'].notnull(), 0)
orderTestData = orderTestData.drop(['temp'], axis=1)
groupedTestMergedData = orderTestData.groupby(['region_id','time_slot','day_of_week'])['answers'].agg('sum').reset_index()
orderTestData = pd.merge(orderTestData,groupedTestMergedData, how='left' , on=['region_id','time_slot','day_of_week'])
print(orderTestData)

                             region_hash  region_id   
0       90c5a34f06ac86aee0fd70e2adce7d8a          1  \
1       90c5a34f06ac86aee0fd70e2adce7d8a          1   
2       90c5a34f06ac86aee0fd70e2adce7d8a          1   
3       90c5a34f06ac86aee0fd70e2adce7d8a          1   
4       90c5a34f06ac86aee0fd70e2adce7d8a          1   
...                                  ...        ...   
555118  1ecbb52d73c522f184a6fc53128b1ea1         66   
555119  1ecbb52d73c522f184a6fc53128b1ea1         66   
555120  1ecbb52d73c522f184a6fc53128b1ea1         66   
555121  1ecbb52d73c522f184a6fc53128b1ea1         66   
555122  1ecbb52d73c522f184a6fc53128b1ea1         66   

                                order_id                         driver_id   
0       31bcca3b90b4d971697156b63900d514  172b54135139345fb9d24d0b23635387  \
1       60d38d302c9689bcd2a31cf236b36c64  9db845353e2256db7ef3d54872d2288a   
2       cca7aca7134cffb46957d59897dcdbbc  773dd49e4148d807b099b42372fd5ac7   
3       85cbab6eaea4b6bf1bc

In [46]:
orderTestData = orderTestData.drop(["answers_x"], axis=1)
orderTestData = orderTestData.rename(columns={"answers_y": "answers"})
# orderTestData.to_csv('mergedOrderData.csv',index=False)

In [47]:
# orderTestData['supply_demand'] = orderTestData['requests'] - orderTestData['answers']
orderTestData['supply_demand'] = orderTestData['requests']
print(orderTestData)

                             region_hash  region_id   
0       90c5a34f06ac86aee0fd70e2adce7d8a          1  \
1       90c5a34f06ac86aee0fd70e2adce7d8a          1   
2       90c5a34f06ac86aee0fd70e2adce7d8a          1   
3       90c5a34f06ac86aee0fd70e2adce7d8a          1   
4       90c5a34f06ac86aee0fd70e2adce7d8a          1   
...                                  ...        ...   
555118  1ecbb52d73c522f184a6fc53128b1ea1         66   
555119  1ecbb52d73c522f184a6fc53128b1ea1         66   
555120  1ecbb52d73c522f184a6fc53128b1ea1         66   
555121  1ecbb52d73c522f184a6fc53128b1ea1         66   
555122  1ecbb52d73c522f184a6fc53128b1ea1         66   

                                order_id                         driver_id   
0       31bcca3b90b4d971697156b63900d514  172b54135139345fb9d24d0b23635387  \
1       60d38d302c9689bcd2a31cf236b36c64  9db845353e2256db7ef3d54872d2288a   
2       cca7aca7134cffb46957d59897dcdbbc  773dd49e4148d807b099b42372fd5ac7   
3       85cbab6eaea4b6bf1bc

In [48]:
orderTestData = orderTestData.drop(['order_id','driver_id','requests','answers'], axis=1)
print(orderTestData)

                             region_hash  region_id                 time   
0       90c5a34f06ac86aee0fd70e2adce7d8a          1  2016-01-23 19:09:39  \
1       90c5a34f06ac86aee0fd70e2adce7d8a          1  2016-01-23 19:29:47   
2       90c5a34f06ac86aee0fd70e2adce7d8a          1  2016-01-23 21:03:43   
3       90c5a34f06ac86aee0fd70e2adce7d8a          1  2016-01-23 07:08:10   
4       90c5a34f06ac86aee0fd70e2adce7d8a          1  2016-01-23 23:11:34   
...                                  ...        ...                  ...   
555118  1ecbb52d73c522f184a6fc53128b1ea1         66  2016-01-31 23:28:02   
555119  1ecbb52d73c522f184a6fc53128b1ea1         66  2016-01-31 17:16:56   
555120  1ecbb52d73c522f184a6fc53128b1ea1         66  2016-01-31 17:06:01   
555121  1ecbb52d73c522f184a6fc53128b1ea1         66  2016-01-31 09:22:35   
555122  1ecbb52d73c522f184a6fc53128b1ea1         66  2016-01-31 15:17:59   

        time_slot  day_of_week        date  supply_demand  
0             116          

In [49]:
orderTestData = pd.merge(orderTestData,poiTestData, how="inner", on='region_id')
print(orderTestData)

                             region_hash  region_id                 time   
0       90c5a34f06ac86aee0fd70e2adce7d8a          1  2016-01-23 19:09:39  \
1       90c5a34f06ac86aee0fd70e2adce7d8a          1  2016-01-23 19:29:47   
2       90c5a34f06ac86aee0fd70e2adce7d8a          1  2016-01-23 21:03:43   
3       90c5a34f06ac86aee0fd70e2adce7d8a          1  2016-01-23 07:08:10   
4       90c5a34f06ac86aee0fd70e2adce7d8a          1  2016-01-23 23:11:34   
...                                  ...        ...                  ...   
555118  1ecbb52d73c522f184a6fc53128b1ea1         66  2016-01-31 23:28:02   
555119  1ecbb52d73c522f184a6fc53128b1ea1         66  2016-01-31 17:16:56   
555120  1ecbb52d73c522f184a6fc53128b1ea1         66  2016-01-31 17:06:01   
555121  1ecbb52d73c522f184a6fc53128b1ea1         66  2016-01-31 09:22:35   
555122  1ecbb52d73c522f184a6fc53128b1ea1         66  2016-01-31 15:17:59   

        time_slot  day_of_week        date  supply_demand  poi_count  
0             11

In [50]:
orderTestData = pd.merge(weatherTestData,orderTestData, how="inner", on=['date','time_slot'])
print(orderTestData)


        weather  time_slot        date                       region_hash   
0             4         43  2016-01-23  90c5a34f06ac86aee0fd70e2adce7d8a  \
1             4         43  2016-01-23  90c5a34f06ac86aee0fd70e2adce7d8a   
2             4         43  2016-01-23  90c5a34f06ac86aee0fd70e2adce7d8a   
3             4         43  2016-01-23  90c5a34f06ac86aee0fd70e2adce7d8a   
4             4         43  2016-01-23  90c5a34f06ac86aee0fd70e2adce7d8a   
...         ...        ...         ...                               ...   
343581        8        141  2016-01-31  1ecbb52d73c522f184a6fc53128b1ea1   
343582        8        141  2016-01-31  1ecbb52d73c522f184a6fc53128b1ea1   
343583        8        141  2016-01-31  1ecbb52d73c522f184a6fc53128b1ea1   
343584        8        141  2016-01-31  1ecbb52d73c522f184a6fc53128b1ea1   
343585        8        141  2016-01-31  1ecbb52d73c522f184a6fc53128b1ea1   

        region_id                 time  day_of_week  supply_demand  poi_count  
0      

In [51]:
mTest = orderTestData.drop_duplicates(subset=['region_id','time_slot','day_of_week','weather','poi_count'], keep='first')
print(mTest)

        weather  time_slot        date                       region_hash   
0             4         43  2016-01-23  90c5a34f06ac86aee0fd70e2adce7d8a  \
55            4         43  2016-01-23  f2c8c4bb99e6377d21de71275afd6cd2   
60            4         43  2016-01-23  58c7a4888306d8ff3a641d1c0feccbe3   
62            4         43  2016-01-23  b26a240205c852804ff8758628c0a86a   
80            4         43  2016-01-23  4b9e4cf2fbdc8281b8a1f9f12b80ce4d   
...         ...        ...         ...                               ...   
343561        8        141  2016-01-31  ba32abfc048219e933bee869741da911   
343571        8        141  2016-01-31  d524868ce69cb9db10fc5af177fb9423   
343572        8        141  2016-01-31  de092beab9305613aca8f79d7d7224e7   
343574        8        141  2016-01-31  bf44d327f0232325c6d5280926d7b37d   
343579        8        141  2016-01-31  1ecbb52d73c522f184a6fc53128b1ea1   

        region_id                 time  day_of_week  supply_demand  poi_count  
0      

<h1 style="background-color:skyblue; color:black">Prediction on Test Data</h1>

In [52]:
# XTest = mTest[['region_id','time_slot','day_of_week','weather','poi_count']]
# YTest = mTest['supply_demand']
# print("accuracy: ",AImodel.score(XTest,YTest))
# print(YTest)

In [53]:
listOfPredictions =  rollTheWindow(orderTestData,100)
maxError= max(listOfPredictions, key=lambda x: x[3])
minEror = min(listOfPredictions, key=lambda x: x[3])
print(f"Max error is {maxError[3]} for window {maxError[0]} to {maxError[1]}")
print(f"Min error is {minEror[3]} for window {minEror[0]} to {minEror[1]}")

Prediction for window [0,100] MSE=1343.73
Prediction for window [100,200] MSE=771.49
Prediction for window [201,301] MSE=848.16
Prediction for window [303,403] MSE=380.76
Prediction for window [406,506] MSE=2844.2
Prediction for window [510,610] MSE=952.82
Prediction for window [615,715] MSE=388.88
Prediction for window [721,821] MSE=649.93
Prediction for window [828,928] MSE=5056.0
Prediction for window [936,1036] MSE=7246.52
Prediction for window [1045,1145] MSE=17956.0
Prediction for window [1155,1255] MSE=5881.18
Prediction for window [1266,1366] MSE=14527.26
Prediction for window [1378,1478] MSE=23824.2
Prediction for window [1491,1591] MSE=37249.0
Prediction for window [1605,1705] MSE=37249.0
Prediction for window [1720,1820] MSE=35404.6
Prediction for window [1836,1936] MSE=5457.68
Prediction for window [1953,2053] MSE=5224.0
Prediction for window [2071,2171] MSE=2357.96
Prediction for window [2190,2290] MSE=8649.0
Prediction for window [2310,2410] MSE=1294.3
Prediction for wind

In [54]:
# print mean of mean squared error
errors = [x[3] for x in listOfPredictions]
print("Mean of mean squared error for above predictions is: ",np.mean(errors))

Mean of mean squared error for above predictions is:  57393.78225543479


In [56]:
Xtest = orderTestData[['region_id','time_slot','day_of_week','weather','poi_count']]
Ytest = orderTestData['supply_demand']
print("Accuracy of the model is: ",AImodel.score(Xtest,Ytest))

Accuracy of the model is:  0.5563403139045251
